In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

C:\Users\Roze\AppData\Local\Temp\ipykernel_8704\4086620180.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [24]:
data = pd.read_csv("users-score-2023.csv", encoding="utf8")
animeData = pd.read_csv("anime-dataset-2023.csv", encoding="utf8")
data.head()
print(type(animeData))

<class 'pandas.core.frame.DataFrame'>


In [3]:
n_users = len(data.user_id.unique())

In [4]:
userAmount = 5000

In [5]:
data_clean = data[data['user_id'] <= userAmount]
data_clean.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5
3,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,304,Aa! Megami-sama! Movie,8


In [6]:
print(data_clean[data_clean['anime_id']  == 35790])

        user_id     Username  anime_id                  Anime Title  rating
5889        208  Arinohyoshi     35790  Tate no Yuusha no Nariagari       8
12410       357       zhambi     35790  Tate no Yuusha no Nariagari       7
17074       426    cnsgurl23     35790  Tate no Yuusha no Nariagari       9
18236       431      izikiel     35790  Tate no Yuusha no Nariagari       7
20266       436      Katsuki     35790  Tate no Yuusha no Nariagari       3
...         ...          ...       ...                          ...     ...
385420     4655      Warzone     35790  Tate no Yuusha no Nariagari       9
389357     4690          TAB     35790  Tate no Yuusha no Nariagari       9
390789     4723      heizzah     35790  Tate no Yuusha no Nariagari       8
391804     4754      burry89     35790  Tate no Yuusha no Nariagari       9
403020     4870      Kayshin     35790  Tate no Yuusha no Nariagari       8

[81 rows x 5 columns]


In [7]:
userRatings = data_clean.pivot_table(index=['user_id'], columns=['anime_id'], values='rating')

In [8]:
userRatings.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,54851,54856,55016,55077,55183,55289,55339,55351,55582,55610
user_id,,,,,,,,,,,,,,,,,,,,,
1,10.0,8.0,10.0,9.0,6.0,7.0,8.0,7.0,8.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,9.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,9.0,8.0,8.0,NaN,NaN,7.0,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
corrMatrix = userRatings.corr()
corrMatrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,54851,54856,55016,55077,55183,55289,55339,55351,55582,55610
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.542113,0.314378,0.040254,0.038047,-0.042461,0.220088,-0.057741,0.100092,0.121094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.320376,NaN,0.268866
5,0.542113,1.000000,0.249916,0.186088,0.296671,0.149810,0.296561,0.099003,0.169657,0.011646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513870,NaN,-0.242536
6,0.314378,0.249916,1.000000,0.120923,0.266598,0.235127,0.156733,0.150342,0.143308,0.080002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.491436,NaN,0.073721
7,0.040254,0.186088,0.120923,1.000000,0.162127,0.258522,0.203410,0.459407,0.073970,0.222612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,1.000000
8,0.038047,0.296671,0.266598,0.162127,1.000000,0.405925,0.309315,0.547981,0.487459,0.429091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
corrMatrix = userRatings.corr(method='pearson', min_periods=100)
corrMatrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,54851,54856,55016,55077,55183,55289,55339,55351,55582,55610
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.542113,0.314378,0.040254,NaN,-0.042461,0.220088,NaN,0.100092,0.121094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.542113,1.000000,0.249916,0.186088,NaN,NaN,0.296561,NaN,0.169657,0.011646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.314378,0.249916,1.000000,0.120923,NaN,0.235127,0.156733,NaN,0.143308,0.080002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.040254,0.186088,0.120923,1.000000,NaN,NaN,0.203410,NaN,NaN,0.222612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# myRatings = userRatings.loc[9522].dropna()
# data_dict = {14813: 10, 37779: 10, 37430: 10, 2566: 2}
# data_dict = {17389: 8, 35790: 8, 1735: 9, 16498: 9}

new_ratings = []
for _ in range(5):  # Assume the user rates 5 animes (adjust as needed)
    anime_id = input("Enter Anime ID: ")
    rating = float(input("Enter Rating (between 1 and 10): "))
    new_ratings.append((int(anime_id), rating))

print(new_ratings)
print(dict(new_ratings))
myRatings = pd.Series(dict(new_ratings))
print(myRatings)
myRatings

[(17389, 8.0), (35790, 8.0), (1735, 9.0), (16498, 9.0), (14813, 2.0)]
{17389: 8.0, 35790: 8.0, 1735: 9.0, 16498: 9.0, 14813: 2.0}
17389    8.0
35790    8.0
1735     9.0
16498    9.0
14813    2.0
dtype: float64


17389    8.0
35790    8.0
1735     9.0
16498    9.0
14813    2.0
dtype: float64

In [12]:
anime_id_to_check = 2566

if anime_id_to_check in corrMatrix.columns:
    print(f"Anime ID {anime_id_to_check} is present in the correlation matrix.")
else:
    print(f"Anime ID {anime_id_to_check} is not present in the correlation matrix.")


Anime ID 2566 is present in the correlation matrix.


In [32]:
simCandidates = pd.Series()
for i in range(1, len(myRatings.index)):
    # print("Adding sims for " + str(myRatings.index[i]) + "...")
    sims = corrMatrix[myRatings.index[i]].dropna()
    # sims = sims.map(lambda x: x * myRatings[i])
    sims = sims.map(lambda x: x * myRatings.loc[myRatings.index[i]])
    # simCandidates = simCandidates.append(sims)
    simCandidates = pd.concat([simCandidates, sims])
    
print("sorting...")
simCandidates.sort_values(inplace= True, ascending= False)
# col = ['anime_id', 'Name', 'Score']
# myDataFrame = simCandidates.to_frame()
# myDataFrame = simCandidates.reset_index().rename(columns={'index': col[0], 0: col[1]})
anime_df = simCandidates.reset_index(name='weight_score')

# Merge with the anime_data DataFrame to retrieve title and score
result_df = pd.merge(anime_df, animeData, how='left', left_on='index', right_on='anime_id')
print(result_df)

# Drop unnecessary columns
result_df = result_df[['anime_id', 'Name', 'Score']]

# Print the resulting DataFrame
result_df

# print(myDataFrame)
# myDataFrame

sorting...
     index  weight_score  anime_id                               Name  \
0     1735      9.000000      1735                 Naruto: Shippuuden   
1    16498      9.000000     16498                 Shingeki no Kyojin   
2       20      5.846533        20                             Naruto   
3    25777      5.696291     25777        Shingeki no Kyojin Season 2   
4    35760      5.610555     35760        Shingeki no Kyojin Season 3   
..     ...           ...       ...                                ...   
683   2164     -0.844070      2164                        Dennou Coil   
684    918     -1.076640       918                            Gintama   
685    323     -1.142971       323                    Mousou Dairinin   
686    323     -1.316741       323                    Mousou Dairinin   
687    486     -1.778965       486  Kino no Tabi: The Beautiful World   

                 English name                  Other name Score  \
0            Naruto Shippuden                

C:\Users\Roze\AppData\Local\Temp\ipykernel_8704\3827658836.py:8: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  simCandidates = pd.concat([simCandidates, sims])


,anime_id,Name,Score
0,1735,Naruto: Shippuuden,8.26
1,16498,Shingeki no Kyojin,8.54
2,20,Naruto,7.99
3,25777,Shingeki no Kyojin Season 2,8.5
4,35760,Shingeki no Kyojin Season 3,8.62
...,...,...,...
683,2164,Dennou Coil,8.04
684,918,Gintama,8.94
685,323,Mousou Dairinin,7.67
686,323,Mousou Dairinin,7.67


In [16]:
anime_id_to_check = 14813

if anime_id_to_check in simCandidates.index:
    print(f"Data with anime_id {anime_id_to_check} is present in simCandidates.")
else:
    print(f"Data with anime_id {anime_id_to_check} is not present in simCandidates.")

found = False
for position, anime_id in enumerate(simCandidates.index, start=1):
    if anime_id == anime_id_to_check:
        found = True
        break

if found:
    print(f"Anime ID {anime_id_to_check} is ranked #{position} in simCandidates.")
else:
    print(f"Anime ID {anime_id_to_check} is not present in simCandidates.")



Data with anime_id 14813 is present in simCandidates.
Anime ID 14813 is ranked #290 in simCandidates.


In [15]:
index_to_check = 1347

try:
    anime_id_at_index = simCandidates.index[index_to_check - 1]
    print(f"The anime ID at index {index_to_check} in simCandidates is {anime_id_at_index}.")
except IndexError:
    print(f"Index {index_to_check} is out of bounds for simCandidates.")


Index 1347 is out of bounds for simCandidates.
